In [ ]:
import numpy as np
import os
import shutil
import glob

In [3]:
def read_points(file_path, dim=4):
    suffix = os.path.splitext(file_path)[1] 
    assert suffix in ['.bin', '.ply']
    if suffix == '.bin':
        return np.fromfile(file_path, dtype=np.float32).reshape(-1, dim)
    else:
        raise NotImplementedError


def write_points(lidar_points, file_path):
    suffix = os.path.splitext(file_path)[1] 
    assert suffix in ['.bin', '.ply']
    if suffix == '.bin':
        with open(file_path, 'w') as f:
            lidar_points.tofile(f)
    else:
        raise NotImplementedError

def drop_points(points, drop_ratio):
    """
    Randomly drops a fraction of points from the input numpy array.

    Args:
        points (np.ndarray): The input array of shape (N, D).
        drop_ratio (float): Fraction of points to drop (between 0 and 1).

    Returns:
        np.ndarray: Array with dropped points.
    """
    N = points.shape[0]
    keep_num = int(N * (1 - drop_ratio))
    indices = np.random.choice(N, keep_num, replace=False)
    return points[indices]

def process_validation_data(validation_file_path, output_file_path, deletion_ratio):
    """
    Reads validation data from validation_file_path and processes each bin file
    before saving the processed data to output_file_path.
    """
    points = read_points(validation_file_path)
    points = drop_points(points, deletion_ratio)
    write_points(points, output_file_path)




In [8]:
def prepare_folder_structure(original_kitti_root, new_kitti_root, ratio):
    folders = ["calib", "image_2", "label_2", "velodyne", "velodyne_reduced"]
    ratio_dir = os.path.join(new_kitti_root, str(ratio))
    os.makedirs(ratio_dir, exist_ok=True)
    pickle_files = glob.glob(os.path.join(original_kitti_root, "*.pkl"))    
    for file_path in pickle_files:
        shutil.copy2(file_path, ratio_dir)
    ratio_dir = os.path.join(ratio_dir, "training")
    os.makedirs(ratio_dir, exist_ok=True)
    for folder in folders:
        folder_path = os.path.join(ratio_dir, folder)
        os.makedirs(folder_path, exist_ok=True)
    return ratio_dir

In [9]:
ratios = [0.2, 0.4, 0.6, 0.8, 0.9]
validation_text = "/home/lucas/repos/PointPillars/pointpillars/dataset/ImageSets/val.txt"
training_bin_file_prefix = "/home/lucas/repos/PointPillars/kitti/training/velodyne_reduced"
with open(validation_text, 'r') as file:
    validation_filenames = file.readlines()
for ratio in ratios:
    output_file_prefix = prepare_folder_structure("/home/lucas/repos/PointPillars/kitti", 
                             "/home/lucas/repos/PointPillars/kitti_deleted", 
                             ratio)
    for file_name in validation_filenames:
        input_path = os.path.join(training_bin_file_prefix, file_name.strip() + ".bin")
        output_path = os.path.join(output_file_prefix,"velodyne_reduced", f"{file_name.strip()}.bin")
        points = process_validation_data(input_path, output_path, ratio)


In [1]:
import pickle
path = "/home/lucas/repos/PointPillars/pointpillars/dataset/ImageSets/val.txt"

with open(path, 'r') as f:
    data = f.readlines()
print(data[805])
    

001610



In [1]:

from pointpillars.dataset import Kitti, get_dataloader
val_dataset = Kitti(data_root="/home/lucas/repos/PointPillars/kitti_deleted/0.6",
                        split='val')
val_dataloader = get_dataloader(dataset=val_dataset, 
                                    batch_size=1, 
                                    num_workers=4,
                                    shuffle=False)


In [ ]:
import torch
from pointpillars.model import PointPillars
import tqdm
data_dict = {}
model = PointPillars(nclasses=3).cuda()
model.load_state_dict(torch.load("pretrained/epoch_160.pth"))
model.eval()
batch_results= None
with torch.no_grad():
    for i, data_dict_original in enumerate(tqdm(val_dataloader)):
        if i == 805:
            for key in data_dict_original:
                for j, item in enumerate(data_dict_original[key]):
                    if torch.is_tensor(item):
                        data_dict_original[key][j] = data_dict_original[key][j].cuda()
            data_dict = data_dict_original
            break
    batched_pts = data_dict['batched_pts']
    batched_gt_bboxes = data_dict['batched_gt_bboxes']
    batched_labels = data_dict['batched_labels']
    batched_difficulty = data_dict['batched_difficulty']
    batch_results = model(batched_pts=batched_pts, 
                    mode='val',
                    batched_gt_bboxes=batched_gt_bboxes, 
                    batched_gt_labels=batched_labels)
    

TypeError: 'module' object is not callable

In [7]:
print(type(batch_results))
print(batch_results)

<class 'list'>
[{'lidar_bboxes': array([[19.579243  , -2.7978206 , -0.8521094 ,  0.6657081 ,  0.69728935,
         1.7805387 , -3.005843  ],
       [ 8.807708  , -3.1225326 , -0.9373024 ,  0.5085847 ,  0.70824975,
         1.7090148 ,  2.6683772 ],
       [ 6.2004294 , -2.8211627 , -1.6241834 ,  1.5427153 ,  3.5344758 ,
         1.4514694 , -1.6109692 ],
       [12.602291  ,  4.224052  , -1.806828  ,  1.5235919 ,  3.578537  ,
         1.4078885 , -1.6159204 ],
       [16.573448  , -2.6436632 , -1.520259  ,  1.5907251 ,  3.5249205 ,
         1.4694097 , -1.5625604 ],
       [21.662922  ,  4.519707  , -1.65463   ,  1.6430291 ,  4.2045426 ,
         1.609315  , -1.5237412 ],
       [35.644783  ,  4.969858  , -1.6814785 ,  1.6142138 ,  4.065442  ,
         1.522496  ,  1.588921  ],
       [32.204247  , -2.514601  , -1.5472792 ,  1.5590453 ,  3.577557  ,
         1.5262825 , -1.6000888 ],
       [51.65022   ,  5.2891026 , -1.8894546 ,  1.5902733 ,  4.1205297 ,
         1.5710205 ,  1.551653

In [ ]:
for j, result in enumerate(batch_results):
    if type(result) != dict:
        print(f"Result {j} is a {type(result)}") 
        print(result)